In [ ]:
import time
import param
import panel as pn
pn.extension()

## Dev experience

### Bind on reference value, not value

Be sure to bind on `param.{parameter}`, not just `{parameter}`. If only on `{parameter}`, it will not trigger an update.

In [ ]:
def show_clicks(clicks):
    return f"Number of clicks: {clicks}"

button = pn.widgets.Button(name="Click me!")
clicks = pn.bind(show_clicks, button.param.clicks)  # not button.clicks!
pn.Row(button, clicks)

### Inherit from `pn.viewer.Viewer`

Instead of inheriting from `param.Parameterized`, opting for inheritance from `pn.viewable.Viewer` allows direct invocation of the class, resembling a native Panel object, i.e. `ExampleApp().servable()` vs `ExampleApp().view().servable()`.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    ...

    def __panel__(self):
        return pn.template.FastListTemplate(
            main=[...],
            sidebar=[...],
        )

ExampleApp().servable();  # semi-colon to suppress output in example

### Build widgets from parameters

To translate parameters into widgets, use `pn.Param`.


In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        return pn.Column(
            pn.Param(self, widgets={"height": pn.widgets.IntInput}),
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

You can also use `from_param` to manually build each component.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    width = param.Integer(default=100, bounds=(1, 200), label="Width of box")
    height = param.Integer(default=100, bounds=(1, 250), label="Height of box")
    color = param.Color(default="red", label="Color of box")

    def __panel__(self):
        width_slider = pn.widgets.IntSlider.from_param(self.param.width)
        height_input = pn.widgets.IntInput.from_param(self.param.height)
        color_picker = pn.widgets.ColorPicker.from_param(self.param.color)
        return pn.Column(
            width_slider,
            height_input,
            color_picker,
            pn.pane.HTML(
                width=self.param.width,
                height=self.param.height,
                styles={"background-color": self.param.color},
            ),
        )


ExampleApp()

### Show templates in notebooks

Templates, at the time of writing, are not able to be rendered properly in Jupyter notebooks.

To continue working with templates in notebooks, call `show` to pop up a new browser window.

In [ ]:
template = pn.template.FastListTemplate(
    main=[...],
    sidebar=[...],
)

# template.show()  # disable showing in examples

### Yield to show intermediate values

Use a generator (yield) to provide incremental updates.

In [ ]:
def increment_to_value(value):
    for i in range(value):
        time.sleep(0.1)
        yield i

slider = pn.widgets.IntSlider(start=1, end=10)
output = pn.bind(increment_to_value, slider.param.value_throttled)
pn.Row(slider, output)

### Watch side effects

For functions that do not return anything (or returns None), be sure to set `watch=True` on `pn.bind` or `pn.depends`.

In [ ]:
def print_clicks(clicks):
    print(f"Number of clicks: {clicks}")

button = pn.widgets.Button(name="Click me!")
pn.bind(print_clicks, button.param.clicks, watch=True)
button

## User Experience

### Update params effectively

Use `obj.param.update`:
- to update multiple params on an object simultaneously
- as a context manager to temporarily set values, restoring original values on completion

In [ ]:
def run(event):
    with progress.param.update(
        bar_color="primary",
        active=True,
    ):
        for i in range(0, 101):
            time.sleep(0.01)
            progress.value = i

button = pn.widgets.Button(name="Run", on_click=run)
progress = pn.indicators.Progress(value=100, active=False, bar_color="dark")
pn.Row(button, progress)

Use `batch_call_watchers` to update multiple params *on separate objects* simultaneously.

In [ ]:
class ExampleApp(pn.viewable.Viewer):

    def __init__(self):
        self._button = pn.widgets.Button(name="Click me", on_click=self._process)
        self._text_1 = pn.widgets.StaticText(value="Hello")
        self._text_2 = self._text_1.clone()

    def _process(self, event):
        with param.parameterized.batch_call_watchers(self):
            self._text_1.value = "Hey"
            self._text_2.value = "Hey"

    def __panel__(self):
        return pn.Row(self._button, self._text_1, self._text_2)


ExampleApp()

### Throttle slider callbacks

To prevent sliders from triggering excessive callbacks, set `throttled=True` so that it only triggers once upon mouse-up.

In [ ]:
pn.extension(throttled=True)

def callback(value):
    return f"# {value}"

slider = pn.widgets.IntSlider(end=10)
output = pn.bind(callback, slider)
pn.Row(slider, output)

Alternatively, limit the scope by binding against `value_throttled` instead of `value`.

In [ ]:
def callback(value):
    return f"# {value}"

slider = pn.widgets.IntSlider(end=10)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

### Defer expensive operations

Start by instantiating the initial layout with placeholder `pn.Columns`, then populate it later in `onload`.

In [ ]:
import time

def onload():
    time.sleep(1)  # simulate expensive operations
    layout.objects = [
        "Welcome to this app!",
    ]

layout = pn.Column("Loading...")
display(layout)
pn.state.onload(onload)

### Show indicator while computing

Set `loading=True` to show a spinner while processing to let the user know it's working.

In [ ]:
import time

def compute(event):
    with layout.param.update(loading=True):
        time.sleep(3)
        layout.append("Computation complete!")

button = pn.widgets.Button(name="Compute", on_click=compute)
layout = pn.Column("Click below to compute", button)

layout

### Manage exceptions gracefully

Use:
- `try` block to update values on success
- `except` block to update values on exception
- `finally` block to update values regardless

In [ ]:
import time

def compute(divisor):
    try:
        busy.value = True
        time.sleep(1)
        output = 1 / divisor
        text.value = "Success!"
    except Exception as exc:
        output = "Undefined"
        text.value = f"Error: {exc}"
    finally:
        busy.value = False
    return f"Output: {output}"

busy = pn.widgets.LoadingSpinner(width=10, height=10)
text = pn.widgets.StaticText()

slider = pn.widgets.IntSlider(name="Divisor")
output = pn.bind(compute, slider)

layout = pn.Column(pn.Row(busy, text), slider, output)
layout

### Cache values for speed

Wrap the decorator `pn.cache` for automatic handling.

In [ ]:
@pn.cache
def callback(value):
    time.sleep(2)    
    return f"# {value}"

slider = pn.widgets.IntSlider(end=3)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

Or, manually handle the cache with `pn.state.cache`.

In [ ]:
def callback(value):
    output = pn.state.cache.get(value)
    if output is None:
        time.sleep(2)
        output = f"# {value}"
        pn.state.cache[value] = output
    return output

slider = pn.widgets.IntSlider(end=3)
output = pn.bind(callback, slider.param.value_throttled)
pn.Row(slider, output)

### Preserve axes ranges on update

To prevent the plot from resetting to its original axes ranges when zoomed in, simply wrap `hv.DynamicMap`.

In [ ]:
import numpy as np
import holoviews as hv
hv.extension("bokeh")

data = []

def add_point(clicks):
    data.append((np.random.random(), (np.random.random())))
    return hv.Scatter(data)

button = pn.widgets.Button(name="Add point")
plot = hv.DynamicMap(pn.bind(add_point, button.param.clicks))
pn.Column(button, plot)

### Reuse objects for efficiency

Imagine Panel components as placeholders and use them as such rather than re-creating them on update.

In [ ]:
import pandas as pd
import numpy as np


def randomize(event):
    df_pane.object = pd.DataFrame(np.random.randn(10, 3), columns=list("ABC"))


button = pn.widgets.Button(name="Compute", on_click=randomize)
df_pane = pn.pane.DataFrame()
button.param.trigger("clicks")  # initialize

pn.Column(button, df_pane)